<a href="https://colab.research.google.com/github/hdorazi/english-phonetics/blob/main/class-2024-fall_1129.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install tensorflow gradio

# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import numpy as np
import gradio as gr
from PIL import Image

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data() # x train: 숫자 imaage, y train: x train image의 라벨 번호
x_train = x_train / 255.0  # Normalize pixel values to [0, 1]
x_test = x_test / 255.0 # model이 훈련이 잘 되었는지 확인하는 용도 (파란점 20개만 따로 빼뒀다가 결과로 나온 모델 즉, y=ax+b가 따로 빼둔 점들과 유사한 위치에 있는지 확인)

# Add a channel dimension (for grayscale images)
x_train = x_train[..., np.newaxis] # input node의 차원에 따라 개수가 달라짐
x_test = x_test[..., np.newaxis]

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    # DNN
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') #sofmax probability 다 더해서 1(확률)로 만들어줌
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test)) # epoch: 데이터를 n번 내보낼 때까지 훈련.

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

# Map digit predictions to word equivalents
digit_to_word = {
    0: "zero",
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine"
}

# Define the Gradio interface function
def classify_digit(data):
    try:
        # Extract composite image from Gradio Sketchpad data
        composite_array = np.array(data["composite"])
        image = Image.fromarray(composite_array)
        image = image.resize((28, 28))  # Resize to match MNIST format
        image_array = np.array(image) / 255.0
        r, g, b, a = image_array[:,:,0], image_array[:,:,1], image_array[:,:,2], image_array[:, :, 3]

        # Make a prediction
        prediction = model.predict(a.reshape(1, 28, 28, 1)).flatten()
        class_idx = np.argmax(prediction)
        confidence = prediction[class_idx]

        # Return the word label and confidence
        return f"{digit_to_word[class_idx]} ({confidence:.2f})"
    except Exception as e:
        return f"Error: {str(e)}"



# Create the Gradio interface
demo = gr.Interface(
    fn=classify_digit,
    inputs=gr.Sketchpad(label="Draw a digit"),  # Use Sketchpad for drawing
    outputs="text",  # Output a text label
    title="Real-time Digit Classifier",
    description="Draw a digit (0-9) on the canvas and see the prediction in words."
)

# Launch the app
demo.launch(debug=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 64s 33ms/step - accuracy: 0.8942 - loss: 0.3331 - val_accuracy: 0.9835 - val_loss: 0.0523
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 59s 31ms/step - accuracy: 0.9853 - loss: 0.0479 - val_accuracy: 0.9857 - val_loss: 0.0465
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 84s 33ms/step - accuracy: 0.9895 - loss: 0.0325 - val_accuracy: 0.9894 - val_loss: 0.0295
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 78s 31ms/step - accuracy: 0.9929 - loss: 0.0231 - val_accuracy: 0.9897 - val_loss: 0.0309
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 88s 34ms/step - accuracy: 0.9954 - loss: 0.0150 - val_accuracy: 0.9919 - val_loss: 0.0267
313/313 - 2s - 8ms/step - accuracy: 0.9919 - loss: 0.0267
Test accuracy: 0.9919
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs.

Created dataset file at: .gradio/flagged/dataset1.csv
